In [2]:
import pandas as pd
import numpy as np
import statistics as stat
import random as random
import math as math

# Ear Phenotypes

In [3]:
useCols = [1, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
df1 = pd.read_csv('../data/earphenotypesformatted.csv', na_values = {'a/n', 'a'}, usecols = useCols)
df2 = pd.read_csv('../data/earphenotypesformatted.csv', na_values = {'a/n', 'a'}, usecols = useCols)
fileList = [df1, df2]
traitList = ['Ear Width', 'Kernel Fill Length', 'Kernel Row Number', 'Kernels per Row', 'Ear Weight', 'Kernel Count', 'Cob Length', 'Cob Width', 'Cob Weight', '100 Kernel weight']

In [4]:
# What if we just concat and pivot
ears = pd.concat([df1, df2])
ears = ears.reset_index(drop=False)
ears = ears.replace({'22.94.':'22.94'})
ears['Kernel Mass'] = ears['Ear Weight'].astype(float) - ears['Cob Weight'].astype(float)
ears['QR Code'].astype(str)
ears['QR Code'] = ears['QR Code'].str.upper()
ears['earNum'] = ears.groupby('QR Code').cumcount()
ears = ears[ears['earNum'] < 4]
plots = pd.pivot(ears, index = 'QR Code', columns = 'earNum')
traitList.append('Kernel Mass')
plots = plots.astype(float)

In [ ]:
plots.dtypes

In [ ]:
# Now that we have our nice data frame, we need to calculate the mean and standard error with all possible sets of 4 and all possible sets of 2 for each trait.
index30 = [1, 2, 3]
index31 = [0, 2, 3]
index32 = [0, 1, 2]
index20 = [0, 1]
index21 = [0, 2]
index22= [0, 3]
index23 = [1, 2]
index24= [1, 3]
index25= [2, 3]
index3List = [index30, index31, index32]
index3ListStr = ['30', '31', '32']
index2List = [index20, index21, index22, index23, index24, index25]
index2ListStr = ['20', '21', '22', '23', '24', '25']

results = pd.DataFrame()
for i in traitList:
    mean = i + 'mean' + '4'
    se = i + 'se' + '4'
    meanSeries = (plots[i][[0, 1, 2, 3]]).mean(axis = 1)
    results[mean] = meanSeries
    seSeries = (plots[i][[0, 1, 2, 3]]).sem(axis = 1)
    results[se] = seSeries
    
    for j in range(3):
        mean = i + 'mean' + index3ListStr[j]
        se = i + 'se' + index3ListStr[j]
        index = index3List[j]
        meanSeries = (plots[i][index]).mean(axis = 1)
        seSeries = (plots[i][index]).sem(axis = 1)
        results[mean] = meanSeries
        results[se] = seSeries
        
    for k in range(6):
        mean = i + 'mean' + index2ListStr[k]
        se = i + 'se' + index2ListStr[k]
        index = index2List[k]
        meanSeries = (plots[i][index]).mean(axis = 1)
        seSeries = (plots[i][index]).sem(axis = 1)
        results[mean] = meanSeries
        results[se] = seSeries

In [ ]:
meanSeries

In [ ]:
results

In [6]:
avgSE = pd.DataFrame()
for t in traitList:
    traitSE2 = t + '.SE.2'
    traitSE3 = t + '.SE.3'
    
    seCols2 = []
    for i in index2ListStr:
        colName = t + 'se' + i
        seCols2.append(colName)
     
    seCols3 = []
    for j in index3ListStr:
        colName = t + 'se' + j
        seCols3.append(colName)
        avgSE[traitSE2] = results[seCols2].mean(axis = 1)
        avgSE[traitSE3] = results[seCols3].mean(axis = 1)
    

In [ ]:
avgSE

In [7]:
corrPerSet = pd.DataFrame()
for t in traitList:
    fullMeanCol = t + 'mean' + '4'
    
    for i in index2ListStr:
        colName = t + 'mean' + i
        cor = results[colName].corr(results[fullMeanCol])
        corrPerSet[colName] = pd.Series()
        corrPerSet.at[0, colName] = cor
    for j in index3ListStr:
        colName = t + 'mean' + j
        cor = results[colName].corr(results[fullMeanCol])
        corrPerSet.at[0, colName] = cor
    

In [ ]:
corrPerSet.columns

In [8]:
summary = pd.DataFrame({'Set Size':[2, 3, 4]})

for t in traitList:
    sem = t + '.MeanSEM'
    corr = t + '.MeanCorr'
    sem2 = t + '.SE.2'
    sem3 = t + '.SE.3'
    semFull = t + 'se4'
    
    meanCols2 = []
    for i in index2ListStr:
        colName = t + 'mean' + i
        meanCols2.append(colName)
        
    meanCols3 = []
    for j in index3ListStr:
        colName = t + 'mean' + j
        meanCols3.append(colName)

    summary.at[0, sem] = avgSE[sem2].mean(axis = 0)
    summary.at[1, sem] = avgSE[sem3].mean(axis = 0)
    summary.at[2, sem] = results[semFull].mean(axis = 0)
    
    summary.at[0, corr] = corrPerSet[meanCols2].mean().values[0]
    summary.at[1, corr] = corrPerSet[meanCols3].mean().values[0]
    summary.at[2, corr] = 1

In [22]:
# Export to csv
summary.to_csv('../analysis/RequiredNumberOfMeasurements.csv', index = False)

# NIR

In [52]:
nir = pd.read_excel('../data/NIRData.xlsx', sheet_name = 'in', dtype = {'Sample ID':str})
nir['protein'] = nir['Protein As is'] / (1 - (nir['Moisture'] / 100))
nir['oil'] = nir['Oil As is'] / ( 1 - (nir['Moisture'] / 100))
nir['fiber'] = nir['Fiber As is'] / (1 - (nir['Moisture'] / 100))
nir['ash'] = nir['Ash As is'] / (1 - (nir['Moisture'] / 100))
nir['starch'] = nir['Starch As is'] / (1 - (nir['Moisture'] / 100))
nir['qr'] = nir['Sample ID'].str.upper()
nir['qr'] = nir['qr'].astype(str)
nir = nir.rename(columns = {'Moisture':'moisture'})
traits = ['starch', 'protein', 'oil', 'fiber', 'ash', 'moisture']
nir = nir[['qr', 'starch', 'protein', 'oil', 'fiber', 'ash', 'moisture']]
# nir = nir[~nir['qr'].str.contains('SYNGENTA 3111')]
# nir = nir[nir[traits] > 0]

In [53]:
nir

,qr,starch,protein,oil,fiber,ash,moisture
0,MV$HYBRID-HIPS$REP2$PLOT66$ROW25$RANGE40$COMME...,76.711890,7.242816,1.944237,-2.189937,0.480718,6.39
1,MV$HYBRID-HIPS$REP2$PLOT66$ROW25$RANGE40$COMME...,68.595304,8.081235,3.903110,0.761582,0.941400,5.46
2,MV$HYBRID-HIPS$REP2$PLOT66$ROW25$RANGE40$COMME...,68.820730,7.625595,3.828662,0.835537,0.930724,5.45
3,MV$HYBRID-HIPS$REP2$PLOT66$ROW25$RANGE40$COMME...,70.262267,7.677665,3.828257,0.824873,0.920051,5.44
4,MV$HYBRID-HIPS$REP2$PLOT66$ROW25$RANGE40$COMME...,68.086459,8.370417,4.598432,0.646323,0.995974,5.62
...,...,...,...,...,...,...,...
8999,SCOTTSBLUFF$HYBRID-HIPS - HIGH NITROGEN$REP2$P...,71.105150,7.682403,4.120172,1.695279,1.051502,6.80
9000,SCOTTSBLUFF$HYBRID-HIPS - HIGH NITROGEN$REP2$P...,72.690676,8.356000,3.416386,1.272984,1.088021,8.09
9001,SCOTTSBLUFF$HYBRID-HIPS - HIGH NITROGEN$REP2$P...,71.479987,8.179736,4.155306,1.243320,1.112444,8.31
9002,SCOTTSBLUFF$HYBRID-HIPS - HIGH NITROGEN$REP2$P...,70.522997,8.339676,4.392737,1.370012,1.152550,8.03
